### Information in this notebook come from (Thanks to Chrus Blake and Alex Krolewski):
README that were the first guide to get the code and then the data was here :
https://data.desi.lbl.gov/desi/users/cblake/lensing/stage0mocks_measurements/akrolew_wp/README.txt

### The code was here: 
 /global/cfs/cdirs/desi/users/akrolew/mock_challenge/wp/config_space_wp_lensing_challenge.py
euivalent web address: https://data.desi.lbl.gov/desi/users/akrolew/mock_challenge/wp/config_space_wp_lensing_challenge.py

### Data is here:
https://data.desi.lbl.gov/desi/users/cblake/lensing/stage0mocks/


In [1]:
import time

from nbodykit.lab import *
from nbodykit.algorithms.pair_counters import *
import numpy as np
import sys
from nbodykit import setup_logging


ImportError: No module named 'nbodykit'

In [3]:
### fits tables to csv files
path='/uufs/chpc.utah.edu/common/home/astro/dawson/sarahE/eboss/'
outpath = '/uufs/chpc.utah.edu/common/home/astro/dawson/sarahE/nbodykit_lab/'

import fitsio
qso,h = fitsio.read(path+'/Mar2020/eBOSS_QSO_NGC_pip_v7_2.dat_z0.8_z2.2_withS_withPIX.fits',header=True)

f = open(outpath+'qsos_ngc_v7_2.dat', 'wb')

w = ((qso['STATUS'] & 2**1) !=0)
q = qso[w]
weight = q['WEIGHT_SYSTOT']*q['WEIGHT_NOZ']*q['WEIGHT_FKP']*q['WEIGHT_CP']
ra = q['RA']
dec = q['DEC']
z = q['Z']






XTENSION= 'BINTABLE'           / binary table extension
BITPIX  =                    8 / array data type
NAXIS   =                    2 / number of array dimensions
NAXIS1  =                  564 / length of dimension 1
NAXIS2  =               229675 / length of dimension 2
PCOUNT  =                    0 / number of group parameters
GCOUNT  =                    1 / number of groups
TFIELDS =                   12 / number of table fields
TTYPE1  = 'PIXEL   '
TFORM1  = 'K       '
TTYPE2  = 's       '
TFORM2  = 'D       '
TTYPE3  = 'EBOSS_TARGET_ID'
TFORM3  = 'K       '
TTYPE4  = 'RA      '
TFORM4  = 'D       '
TTYPE5  = 'DEC     '
TFORM5  = 'D       '
TTYPE6  = 'Z       '
TFORM6  = 'D       '
TTYPE7  = 'WEIGHT_SYSTOT'
TFORM7  = 'D       '
TTYPE8  = 'WEIGHT_CP'
TFORM8  = 'D       '
TTYPE9  = 'WEIGHT_NOZ'
TFORM9  = 'D       '
TTYPE10 = 'WEIGHT_FKP'
TFORM10 = 'D       '
TTYPE11 = 'STATUS  '
TFORM11 = 'J       '
TTYPE12 = 'WEIGHT_BW'
TFORM12 = '60K     '

In [ ]:
t0 = time.time()

setup_logging() # log output to stdout

Rmin = float(sys.argv[1])
Rmax = float(sys.argv[2])
Nbins = int(sys.argv[3]) # Note that this is actually the number of *boundary points*--if I want 30 bins, I need nbins=31
flag = sys.argv[4]

#Rmax = 10.
#Nbins = 11

print(time.time()-t0)

# Files
#input_path = '/global/project/projectdirs/desi/mocks/bgs/MXXL/MXXL_BGS_HOD/'
#input_file = 'periodic_box_z0.2.dat'

#input_path = '/global/project/projectdirs/desi/users/caguayo/PMILL_ELG_mocks/'
#input_file = 'DESI_ELG_z0.76_catalogue.dat'
#BoxSize = 542.16

#input_path = '/global/cfs/cdirs/desi/mocks/UNIT/HOD_Shadab/HOD_boxes/redshift0.9873/'
#input_file = 'UNIT_DESI_Shadab_HOD_snap97_ELG_v0.txt'
#BoxSize = 1000

# Files
#input_path = '/project/projectdirs/desi/users/cblake/lensing/stage0mocks/'
input_path = '/uufs/chpc.utah.edu/common/home/astro/dawson/sarahE/nbodykit_lab/'
name_data = 'stage0mock_reg1_lenses_BGS_zs0pt1_0pt3.dat'
name_random = 'stage0mock_reg1_randlenses_BGS_zs0pt1_0pt3.dat'

binning = 'log'
pimax = 80
P0 = 5000

output_path = '/global/cfs/cdirs/desi/users/akrolew/mock_challenge/'
#output_subdir = 'FS_Krolewski_MockChallenge_1/'
output_name_xi = 'xi2D_' + name_data.split('.dat')[0] + '.txt'
output_name_xi_l = 'xil' + name_data.split('.dat')[0] + '.txt'
output_name_wp = 'wp_pimax_%.1f_' % (pimax) + name_data.split('.dat')[0] + '.txt'

names = ['RA','DEC','Z_rsd','Weight']

data_tmp = np.loadtxt(input_path + name_data,skiprows=3)
rand_tmp = np.loadtxt(input_path + name_random,skiprows=3)
np.savetxt(output_path + 'data_tmp.txt',data_tmp)
np.savetxt(output_path + 'rand_tmp.txt',rand_tmp)

# Read file
data = CSVCatalog(output_path + 'data_tmp.txt', names)
randoms = CSVCatalog(output_path + 'rand_tmp.txt', names)

# Fiducial cosmology to convert angles to positions
# from /global/homes/c/chuang/desi_projectdir_UNIT/HOD_Shadab/single_snapshot_lightcone/red0.8_1.2_sub5/params_Planck15Table4LastColumn.ini
cosmo = cosmology.Cosmology(h=0.6774).match(Omega0_m=0.286)

# add Cartesian position column
#data['Position'] = transform.SkyToCartesian(data['ra'], data['dec'], data['z_rsd'], cosmo=cosmo)
#randoms['Position'] = transform.SkyToCartesian(randoms['ra'], randoms['dec'], randoms['z_rsd'], cosmo=cosmo)

epsilon = 1e-4
#edges = np.linspace(0,Rmax,Nbins) # I think that 128 tasks on 4 nodes is sufficient. Need to test. Got 160 to work with 3 nodes in abt 100 s
# frankly I'm baffled by this: still takes far too long with rmax = 50, 256 tasks on 8 nodes.
# >= 20 mins
# Let's try rmax = 10 on 256 tasks....
# Then we can try running only part of the code... 
# OK, it took 40 s to get out to 10...
#edges[0] = epsilon # Won't accept 0 as the first bin (no self-pairs)
edges = np.logspace(np.log10(Rmin),np.log10(Rmax),Nbins)
t0 = time.time()

#data['Weight'] = 1./(1 + data['nz'] * P0)
#randoms['Weight'] = 1./(1 + randoms['nz'] * P0)

if flag == 'xi':
        pc = SurveyData2PCF('2d',data,randoms,edges,cosmo=cosmo,Nmu=120,ra='RA',dec='DEC',redshift='Z_rsd',show_progress=True,nthreads=100,xbin_refine_factor=2,ybin_refine_factor=2,zbin_refine_factor=2,weight='Weight')

        mus = np.linspace(0.5 * (1./120.), 0.5 * (119./120. + 1.),120)

        f = open(output_path + output_name_xi,'w')
        f.write('# Krolewski mock challenge 1, input from ' + input_path + name_data + '\n')
        f.write('# Code to generate this measurement in ' + __file__ + '\n')
        #f.write('# Boxsize = %.1f\n'  % BoxSize)
        f.write('# Binning = ' + binning + '\n')
        f.write('# r mu xi DD\n')
        for i in range(pc.corr['r'].shape[0]):
	        for j in range(pc.corr['r'].shape[1]):
		        #print(mus[j])
		        f.write('%20.8e %20.8e %20.8e %i\n' % (pc.corr['r'][i][j], mus[j], pc.corr['corr'][i][j], pc.D1D2['npairs'][i][j]))
	
        f.close()

        print(time.time()-t0)

        # In sim box we can compute multipoles from FFTCorr
        #Nmesh = 1024
        #mesh = cat.to_mesh(window='tsc',Nmesh=Nmesh,compensated=True,interlaced=True,position='RSDPosition')
        #r = FFTCorr(mesh,mode='2d',dr=1.0,rmin=0.0,rmax=200.0,Nmu=120,los=[0,0,1],poles=[0,2,4])
        # But better to compute them using "to_poles" so that we can do this for data as well

        poles = pc.corr.to_poles([0,2,4])

        f = open(output_path + output_name_xi_l,'w')
        f.write('# Krolewski mock challenge 1, input from ' + input_path + name_data + '\n')
        f.write('# Code to generate this measurement in ' + __file__ + '\n')
        #f.write('# Boxsize = %.1f\n'  % BoxSize)
        f.write('# Binning = ' + binning + '\n')
        f.write('# r xi_0 xi_2 xi_4\n')
        for i in range(poles['r'].shape[0]):
	        f.write('%20.8e %20.8e %20.8e %20.8e\n' % (poles['r'][i], poles['corr_0'][i], poles['corr_2'][i], poles['corr_4'][i]))
	
        f.close()

        print(time.time()-t0)
if flag == 'wp':
        pc_proj = SurveyData2PCF('projected',data,randoms,edges,cosmo=cosmo,ra='RA',dec='DEC',redshift='Z_rsd',show_progress=True,nthreads=100,xbin_refine_factor=2,ybin_refine_factor=2,zbin_refine_factor=2,pimax=pimax,weight='Weight')

        f = open(output_path + output_name_wp,'w')
        f.write('# Krolewski mock challenge 0, input from ' + input_path + name_data + '\n')
        f.write('# Code to generate this measurement in ' + __file__ + '\n')
        #f.write('# Boxsize = %.1f\n'  % BoxSize)
        f.write('# Binning = ' + binning + '\n')
        f.write('# pi_max = %.1f\n' % pimax)
        f.write('# Uniform weighting\n')
        f.write('# Om = 0.286, h = 0.6774\n')
        f.write('# rp wp\n')
        for i in range(pc_proj.wp['rp'].shape[0]):
                f.write('%20.8e %20.8e\n' % (pc_proj.wp['rp'][i], pc_proj.wp['corr'][i]))

        f.close()